In [1]:
import arcpy.management
from arcgis.gis import GIS
from arcgis.raster import ImageryLayer
from arcgis.raster.functions import apply


import requests
import zipfile
import os
import shutil
import tempfile
import datetime
import time
import arcpy
import hashlib
import arcpy

from getpass import getpass

C:\Users\rami8629\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Functions
<font color="red">**CAUTION**: Modify the functions below only if you understand the consequences.</font>

In [2]:

def calculate_size(cell_size, max_pixels):
    # Calculate the size in square meters
    size_sqm = (cell_size * max_pixels)**2

    # Convert the size to square kilometers
    size_sqkm = size_sqm / 1e6

    return size_sqkm
def create_tessellation(aoi, cell_size, max_pixels):

    # Create the tessellation
    tessellation = arcpy.GenerateTessellation_management(
        Output_Feature_Class=r"memory/tessellation",
        Extent=aoi,
        Shape_Type="SQUARE",
        Size =  f"{calculate_size(cell_size, max_pixels)} SquareKilometers"

    )

    # Initialize the list of extents
    extents = []

    # Assume tessellation is the output from the create_tessellation function
    with arcpy.da.SearchCursor(tessellation, ["SHAPE@"]) as cursor:
        for row in cursor:
            feature = row[0]
            extent = feature.extent

            # Add the extent to the list
            extents.append(f'{extent.XMin} {extent.YMin} {extent.XMax} {extent.YMax}')

    return extents



# def download_image(username, password, max_pixels, extent, output_dir):
#     gis = GIS("https://www.arcgis.com", username, password)
#     sentinel_item=gis.content.search('58a541efc59545e6b7137f961d7de883', 'Imagery Layer', outside_org=True)[0]
#     s2_layer = sentinel_item.layers[0]

#     elevation_allbands = apply(s2_layer, 'None')

#     # Generate a unique hash for the extent
#     extent_hash = hashlib.md5(str(extent).encode()).hexdigest()
    
#     # Define the output filename
#     out_file = f"elevation_{extent_hash}.tif"

#     if out_file in os.listdir(output_dir):
#         return
    
#     # Export the image
#     elevation_allbands.export_image(bbox=extent, 
#                         save_folder=output_dir, 
#                         save_file=out_file, 
#                         image_sr=3857, 
#                         bbox_sr=3857, 
#                         size=[max_pixels, max_pixels],
#                         f="image", 
#                         export_format="tiff")


def download_image(extent, cell_size, pixel_count, output_dir, i):    
    # Create a raster layer from the image
    raster_layer = arcpy.MakeImageServerLayer_management(in_image_service="https://elevation.arcgis.com/arcgis/rest/services/WorldElevation/Terrain/ImageServer", 
                                                    out_imageserver_layer= f"raster_layer_{i}", processing_template="None")

    # Split the raster
    with arcpy.EnvManager(extent=f'{extent} GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'):
        arcpy.SplitRaster_management(in_raster=raster_layer, out_folder=output_dir, out_base_name=f"elevation_{i}_", 
                                    split_method="SIZE_OF_TILE", format="TIFF", cell_size=f'{cell_size} {cell_size}', tile_size=f'{pixel_count} {pixel_count}')
    arcpy.Delete_management(raster_layer)

def process_raster_data(output_dir):
    with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'):
        outgdb = os.path.join(output_dir, "elevation.gdb")
        arcpy.management.CreateFileGDB(output_dir, "elevation.gdb")

        arcpy.management.CreateMosaicDataset(
            in_workspace=outgdb,
            in_mosaicdataset_name="elevation",
            coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
            num_bands=None,
            pixel_type="",
            product_definition="",
            product_band_definitions=None
        )
        arcpy.management.AddRastersToMosaicDataset(
            in_mosaic_dataset=os.path.join(outgdb, "elevation"),
            raster_type="Raster Dataset",
            input_path=f"{output_dir}",
            update_cellsize_ranges="UPDATE_CELL_SIZES",
            update_boundary="UPDATE_BOUNDARY",
            update_overviews="UPDATE_OVERVIEWS",
            maximum_pyramid_levels=None,
            maximum_cell_size=0,
            minimum_dimension=1500,
            spatial_reference=None,
            filter="",
            sub_folder="SUBFOLDERS",
            duplicate_items_action="ALLOW_DUPLICATES",
            build_pyramids="BUILD_PYRAMIDS",
            calculate_statistics="CALCULATE_STATISTICS",
            build_thumbnails="BUILD_THUMBNAILS",
            operation_description="",
            force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
            estimate_statistics="ESTIMATE_STATISTICS",
            aux_inputs=None,
            enable_pixel_cache="NO_PIXEL_CACHE",
        )
    return os.path.join(outgdb, "sentinel2")


# Variables

In [3]:

# Define Area of Interest (AOI) and output directory
# Change the aoi to the path of the feature class or shapefile that represents the area of interest
aoi = r"E:\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\GeoAi & Deep Learning\Demos\Biomass_Cantabria\Biomass_Cantabria.gdb\Cantabria_Boundaries_Project"
# Change the output_dir to the directory where you want to save the downloaded images and the processed raster data
output_dir = r"E:\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\GeoAi & Deep Learning\Demos\Biomass_Cantabria\elevation_data"
arcpy.env.overwriteOutput = True
#Input AGOL credentials
username = input("Enter your ArcGIS Online username: ")
password = getpass("Enter your ArcGIS Online password: ")
# Sign in to ArcGIS
arcpy.SignInToPortal("https://www.arcgis.com", username, password)


# Define the maximum number of pixels
pixel_count = 4000
cell_size = 10

# Download and create mosaic

**DISCLAIMER**: Running the following cell may take <font color="green">**minutes to hours to sometimes days**</font> depending on the size of your <font color="green">**Area of Interest (AOI)**</font> and the range of your <font color="green">**start and end dates**</font>. Please be patient and allow the process to complete.

In [4]:

extents = create_tessellation(aoi, cell_size, pixel_count)

total_images = len(extents)
print(f"Total images to download: {total_images}")

downloaded_images = 0

for i, extent in enumerate(extents):
    # Create a subdirectory in the output directory for the current extent
    sub_output_dir = os.path.join(output_dir, f"extent_{i}")
    os.makedirs(sub_output_dir, exist_ok=True)
    
    download_image(extent, cell_size, pixel_count, sub_output_dir, i)
    downloaded_images += 1
    print(f"Downloaded {downloaded_images} of {total_images}, {total_images - downloaded_images} left")

    # Move all files from the subdirectory to the output directory
    for file_name in os.listdir(sub_output_dir):
        shutil.move(os.path.join(sub_output_dir, file_name), output_dir)
    
    # Delete the subdirectory
    shutil.rmtree(sub_output_dir)

print(f"Downloaded Rasters saved to {output_dir}")

# Process the raster data
out_mosaic = process_raster_data(output_dir)

print(f"Processed raster data saved to {out_mosaic}")

Total images to download: 12
